In [ ]:
import sys
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
sys.path.append('../../')

import numpy as np
import random
from IPython.display import Image

import torch
from torch.utils.data import DataLoader

from torchvision.datasets import CIFAR10

from utils import show_first_batch, transform_data, show_images, plot_metrics_iddpm, show_tensor_images
from train import Trainer
from diffusion_models.iddpm import GaussianDiffusion
from metrics.fid_score import fid_score
from metrics.inception_score import inception_score
from diffusion_models.iddpm import Classifier, classifier_cond_fn

from reverse_models.iunet.unet import Unet

import ipywidgets as widgets
widgets.IntSlider()

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
dataset = transform_data(CIFAR10, store_path="../../datasets")
train_dataloader = DataLoader(dataset, batch_size=128, num_workers=0, shuffle=True)
show_first_batch(train_dataloader)

# Linear scheduler

In [ ]:
model = Unet(dim = 64, channels=1).to(device)

diffusion = GaussianDiffusion(
    model,
    image_size = 32,
    timesteps = 1000,           
    loss_type = 'l2'  
    beta_schedule="linear",
).to(device)

In [ ]:
trainer = Trainer(
    diffusion,
    train_dataloader,
    train_lr = 2e-4,
    train_num_steps = 700000, 
    device=device
) 

In [ ]:
trainer.train()

In [ ]:
generated_images = diffusion.sample(batch_size = 128, return_all_timesteps=True).cpu()


In [ ]:
plot_metrics_iddpm(generated_images, dataset, device=torch.device('cpu'), n_timestamps=1000, batch_size=128)

In [ ]:
generated_images_for_show = diffusion.sample(batch_size = 128).cpu()
show_images(generated_images_for_show)

# Cosine scheduler

In [ ]:
model = Unet(dim = 64, channels=1).to(device)

diffusion = GaussianDiffusion(
    model,
    image_size = 32,
    timesteps = 1000,           
    loss_type = 'l2'  
    beta_schedule="cosine",
).to(device)

In [ ]:
trainer = Trainer(
    diffusion,
    train_dataloader,
    train_lr = 2e-4,
    train_num_steps = 700000, 
    device=device
) 

In [ ]:
trainer.train()

In [ ]:
generated_images = diffusion.sample(batch_size = 128, return_all_timesteps=True).cpu()

In [ ]:
plot_metrics_iddpm(generated_images, dataset, device=torch.device('cpu'), n_timestamps=1000, batch_size=128)

In [ ]:
generated_images_for_show = diffusion.sample(batch_size = 128).cpu()
show_images(generated_images_for_show)

# Sigmoid scheduler

In [ ]:
model = Unet(dim = 64, channels=1).to(device)

diffusion = GaussianDiffusion(
    model,
    image_size = 32,
    timesteps = 1000,           
    loss_type = 'l2'  
    beta_schedule="sigmoid",
).to(device)

In [ ]:
trainer = Trainer(
    diffusion,
    train_dataloader,
    train_lr = 2e-4,
    train_num_steps = 700000, 
    device=device
) 

In [ ]:
trainer.train()

In [ ]:
generated_images = diffusion.sample(batch_size = 128, return_all_timesteps=True).cpu()

In [ ]:
plot_metrics_iddpm(generated_images, dataset, device=torch.device('cpu'), n_timestamps=1000, batch_size=128)

In [ ]:
generated_images_for_show = diffusion.sample(batch_size = 128).cpu()
show_images(generated_images_for_show)